### Text Generation with Pretrained NanoGPT

This notebook shows how to generate text using a pretrained NanoGPT model.

By default:

1. The pretrained model and metadata are loaded from the `baselines/nanogpt/shakespeare-char` folder.
2. Generation starts from a configurable prompt (e.g., "To be, or not to be...").

Loading libraries and setting up the environment

In [1]:
import sys
import os

root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)

In [2]:
import torch
import pickle
from contextlib import nullcontext
from baselines.nanogpt.model import GPT, GPTConfig

Configuring the model for sampling

In [7]:
#? Maybe we can have a config.py file to avoid hardcoding paths
# MODEL_PATH = root_path + "/baselines/nanogpt/shakespeare-char/baseline_model.pt"
# META_PATH = root_path + "/baselines/nanogpt/shakespeare-char/meta.pkl"
from config import MODEL_PATH, META_PATH



START_PROMPT = "to be, or not to be -that is the question:\n"
NUM_SAMPLES = 1
MAX_NEW_TOKENS = 500
TEMPERATURE = 0.8
TOP_K = 200
SEED = 42
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
COMPILE = False

In [8]:
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

device_type = 'cuda' if DEVICE == 'cuda' else 'cpu'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[DTYPE]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

Loading the model and tokenizer

In [9]:
# Loading model
checkpoint = torch.load(MODEL_PATH, map_location='cpu')
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)

# Remove unwanted prefixes if any
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k, v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)
model.eval().to(DEVICE)

if COMPILE:
    model = torch.compile(model)

# Loading tokenizer
with open(META_PATH, 'rb') as f:
    meta = pickle.load(f)

stoi, itos = meta['stoi'], meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


number of parameters: 10.65M


Example: Shakespeare text generation

In [11]:
START_PROMPT = "to be, or"

In [12]:
# Encoding prompt 
if START_PROMPT.startswith("FILE:"):
    with open(START_PROMPT[5:], 'r', encoding='utf-8') as f:
        START_PROMPT = f.read()

start_ids = encode(START_PROMPT)
x = torch.tensor(start_ids, dtype=torch.long, device=DEVICE)[None, ...]

# Generating text samples
with torch.no_grad():
    for i in range(NUM_SAMPLES):
        y = model.generate(x, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, top_k=TOP_K)
        print(f"--- Sample {i+1} ---")
        print(decode(y[0].tolist()))
        print()

--- Sample 1 ---
to be, or the king-confess;
And therefore a maid be speak in the certain
To mine solemn the war king many of Hermio?

JOHN OF GAUNT:
A prisoner day, that same hath been
That dargerous nighter, of it is now to-morrow
The force of the ladies of Rome, and of which say
That she were to Could have so other of might
To see the base. What pleasure is the bear
More?

POLIXENES:
Tull I call to his daughter.

MENENIUS:
Well, I must prithee to thee,
If it is in me country's sadness.

COMINIUS:
Come, you have idle t

